# Rectangle Algebra

In [63]:
import qualreas as qr
import os
import numpy as np

path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

## Load the 2D Point Algebra

The 2-dimensional point algebra is based on the eight compass directions, N, S, E, W, NW, NE, SW, SE, and EQ (equals), and it is derived in the second half of the Jupyter notebook, "derive_point_algebras.ipynb".

In [64]:
ptalg2d = qr.Algebra(os.path.join(path, "Algebras/2D_Point_Algebra.json"))

In [65]:
ptalg2d.summary()

  Algebra Name: 2D_Point_Algebra
   Description: Autogenerated 2-dimensional point algebra
 Equality Rels: EQ
     Relations:
            NAME (SYMBOL)         CONVERSE (ABBREV)  REFLEXIVE  SYMMETRIC TRANSITIVE   DOMAIN        RANGE
              South (  S)               North (  N)    False      False       True       2DPt          2DPt
             Equals ( EQ)              Equals ( EQ)     True       True       True       2DPt          2DPt
              North (  N)               South (  S)    False      False       True       2DPt          2DPt
          Southwest ( SW)           Northeast ( NE)    False      False       True       2DPt          2DPt
               West (  W)                East (  E)    False      False       True       2DPt          2DPt
          Northwest ( NW)           Southeast ( SE)    False      False       True       2DPt          2DPt
          Southeast ( SE)           Northwest ( NW)    False      False       True       2DPt          2DPt
           

In [66]:
qr.print_point_algebra_composition_table(ptalg2d)

2D_Point_Algebra
Elements: S, EQ, N, SW, W, NW, SE, E, NE
 rel1 ; rel2 = composition
   S      S      S
   S     EQ      S
   S      N      S|EQ|N
   S     SW      SW
   S      W      SW
   S     NW      SW|W|NW
   S     SE      SE
   S      E      SE
   S     NE      SE|E|NE
------------------------------
  EQ      S      S
  EQ     EQ      EQ
  EQ      N      N
  EQ     SW      SW
  EQ      W      W
  EQ     NW      NW
  EQ     SE      SE
  EQ      E      E
  EQ     NE      NE
------------------------------
   N      S      S|EQ|N
   N     EQ      N
   N      N      N
   N     SW      SW|W|NW
   N      W      NW
   N     NW      NW
   N     SE      SE|E|NE
   N      E      NE
   N     NE      NE
------------------------------
  SW      S      SW
  SW     EQ      SW
  SW      N      SW|W|NW
  SW     SW      SW
  SW      W      SW
  SW     NW      SW|W|NW
  SW     SE      S|SW|SE
  SW      E      S|SW|SE
  SW     NE      S|EQ|N|SW|W|NW|SE|E|NE
------------------------------
   W      S

## Derive Rectangle Relations

### Define a Four Point Network (Class) for 2D Points

In [67]:
class FourPointNet(qr.Network):
    """Create four Temporal Entities that represent spatio-temporal points,
    and use them to express two independent entities. For example, (s1,e1)
    and (s2,e2), where s1 < e1 and s2 < e2, represents two proper intervals.
    Using '<\|=' instead of '<', would represent two intervals where one or
    both might be points.  Return the network and the four spatio-temporal
    entities.
    """
    def __init__(self, algebra, name, entity_class, entity_name,
                 lessthanstr, startname="StartPt", endname="EndPt"):
        self.algebra = algebra
        self.lessthan = algebra.relset(lessthanstr)
        # Start & End Points of entity 1
        s1 = startname + "1"
        e1 = endname + "1"
        self.start1 = entity_class([entity_name], name=s1)
        self.end1 = entity_class([entity_name], name=e1)
        # Start & End Points of entity 2
        s2 = startname + "2"
        e2 = endname + "2"
        self.start2 = entity_class([entity_name], name=s2)
        self.end2 = entity_class([entity_name], name=e2)
        self.name_list = [s1, e1, s2, e2]
        super().__init__(algebra, name)
        self.add_constraint(self.start1, self.end1, self.lessthan, verbose=False)
        self.add_constraint(self.start2, self.end2, self.lessthan, verbose=False)

    def get_points(self):
        """Return a list of the 4 spatio-temporal entities that correspond to the
        start point #1, end point #1, start point #2, and end point #2.
        """
        return [self.start1, self.end1, self.start2, self.end2]

    def __ontology_classes(self, start, end):
        """The constraints between the start and end points of a spatio-temporal
        entity determine whether it belongs to the class of Point, ProperIntervals, or
        both.  Return a list containing the class names for the input network, net."""
        classes = set()
        constr = self.edges[start, end]['constraint']
        # Recall, constraints are bitsets, and because we want this method to
        # work generally for many different types of algebras (e.g., interval,
        # rectangle) we need to convert the bitset into a simple tuple of strings,
        # using the bitset method, members, otherwise the 'in' tests below won't
        # work generally.
        constraints = constr.members()

        # Interval Algebras
        if '=' in constraints:
            classes.add('Point')
        if '<' in constraints:
            classes.add('ProperInterval')

        # 2D Rectangle Algebras
        if 'EQ' in constraints:
            classes.add('2DPoint')
        if 'SW' in constraints:
            classes.add('ProperRectangle')
        if 'W' in constraints:
            classes.add('2DSegment')
        if 'S' in constraints:
            classes.add('2DSegment')
        
        return list(classes)

    def domain_and_range(self):
        """Return a tuple, (domain, range), for the interval/point/rectangle
        relation represented by the input 4-point network."""
        return (self.__ontology_classes(self.start1, self.end1),
                self.__ontology_classes(self.start2, self.end2))


In [80]:
# four_pt_net = FourPointNet(ptalg2d, "FourPtNet", qr.SpatialEntity, "2DPoint",
#                            ["EQ", "W", "S", "SW"], startname="LL", endname="UR")

four_pt_net = FourPointNet(ptalg2d, "FourPtNet", qr.SpatialEntity, "2DPoint",
                           ["SW"], startname="LL", endname="UR")

four_pt_net

The following few cells are just checks to make sure the the four point network looks OK.

In [81]:
four_pt_net.get_points()

[SpatialEntity(['2DPoint'] 'LL1'),
 SpatialEntity(['2DPoint'] 'UR1'),
 SpatialEntity(['2DPoint'] 'LL2'),
 SpatialEntity(['2DPoint'] 'UR2')]

In [82]:
four_pt_net.domain_and_range()

(['ProperRectangle'], ['ProperRectangle'])

In [83]:
four_pt_net.summary()


FourPtNet: 4 nodes, 8 edges
  Algebra: 2D_Point_Algebra
  LL1:['2DPoint']
    => LL1: EQ
    => UR1: SW
  UR1:['2DPoint']
    => UR1: EQ
  LL2:['2DPoint']
    => LL2: EQ
    => UR2: SW
  UR2:['2DPoint']
    => UR2: EQ


In [84]:
four_pt_net.propagate()

True

In [85]:
four_pt_net.summary()


FourPtNet: 4 nodes, 16 edges
  Algebra: 2D_Point_Algebra
  LL1:['2DPoint']
    => LL1: EQ
    => UR1: SW
    => LL2: S|EQ|N|SW|W|NW|SE|E|NE
    => UR2: S|EQ|N|SW|W|NW|SE|E|NE
  UR1:['2DPoint']
    => UR1: EQ
    => LL2: S|EQ|N|SW|W|NW|SE|E|NE
    => UR2: S|EQ|N|SW|W|NW|SE|E|NE
  LL2:['2DPoint']
    => LL2: EQ
    => UR2: SW
  UR2:['2DPoint']
    => UR2: EQ


## Derive the Consistent Networks for Proper Rectangles

In [86]:
signature_name_mapping_dict = qr.signature_name_mapping

def signature_name_mapping(key):
    return signature_name_mapping_dict[key]

def generate_consistent_networks(point_algebra, entity_class, entity_name,
                                 lessthan="<", startname="StartPt", endname="EndPt",
                                 verbose=False):
    """For a given point algebra and less than relation, derive all possible consistent networks
    of 4 points, where the points represent the start and end points of 2 intervals."""
    consistent_nets = dict()
    for elem13 in point_algebra.elements:
        for elem23 in point_algebra.elements:
            for elem14 in point_algebra.elements:
                for elem24 in point_algebra.elements:
                    four_pt_net_name = elem13 + ',' + elem14 + ',' + elem23 + ',' + elem24
                    ptnet = FourPointNet(point_algebra, four_pt_net_name,
                                         entity_class, entity_name,
                                         lessthan, startname, endname)
                    pt1, pt2, pt3, pt4 = ptnet.get_points()
                    rs13 = point_algebra.relset(elem13)
                    rs14 = point_algebra.relset(elem14)
                    rs23 = point_algebra.relset(elem23)
                    rs24 = point_algebra.relset(elem24)
                    ptnet.add_constraint(pt1, pt3, rs13)
                    ptnet.add_constraint(pt1, pt4, rs14)
                    ptnet.add_constraint(pt2, pt3, rs23)
                    ptnet.add_constraint(pt2, pt4, rs24)
                    if ptnet.propagate():
                        elem_key = ",".join([str(rs13), str(rs14), str(rs23), str(rs24)])
                        consistent_nets[signature_name_mapping(elem_key)] = ptnet
                        if verbose:
                            print("==========================")
                            if elem_key in signature_name_mapping:
                                print(elem_key)
                                print(signature_name_mapping(elem_key))
                            else:
                                print("UNKNOWN")
                            print(ptnet.domain_and_range())
                            print(np.array(ptnet.to_list()))
    print(f"\n{len(consistent_nets)} consistent networks")
    return consistent_nets

When we run the code below (with lessthan=["SW"]) then we will get 169 (i.e., 13x13) consistent 4 point networks, <b>and we do!</b>

In that case, each network corresponds to a unique configuration of two <b>proper</b> rectangles.

By <b>proper rectangle</b> we mean that the rectangles cannot degenerate into segments or points.

If we set lessthan=["W", "SW"], then line segments (0 height) can be considered to be rectangles, and we will get 234 configurations.  If we use "S", instead of "W", then 0-width line segments can be considered to be rectangles, and we will again get 234 configurations.

If lessthan=["EQ", "W", "SW"], then points can be considered to be (degenerate) rectangles, and we will get 261 configurations.

lessthan=["EQ", "W", "S", "SW"] yields 324 configurations of two (possibly degenerate) rectangles.

In [87]:
result_proper = generate_consistent_networks(ptalg2d, qr.SpatialEntity, "2DPoint",
                                             lessthan=["SW"], startname="LL", endname="UR",
                                             verbose=False)

KeyError: 'S,SW,SE,S'

In [ ]:
result_all = generate_consistent_2D_networks(ptalg2d, lessthan=["EQ", "W", "S", "SW"], verbose=False)

In [ ]:
len(result_all)

## Assigning Names to the Rectangle Relations

To understand the naming scheme for rectangle relations, imagine the two rectangles being projected, as intervals, onto two 1-dimensional axes, one axis horizontal and the other axis vertical. Then obtain the names of the interval relations depicted on the two axes and concatenate them, horizontal-name:vertical-name, e.g., D:O (during:overlaps). For proper intervals, that will produce the $13^2=169$ rectangle relation names for the <i>result_proper</i> rectangle relations derived above. See the hand-drawn figure, farther below, for an illustration of this.

Then, it remains to map the each of the Four2DPointNet objects onto the proper rectangle name. This is done by mapping the compass directions (N, NW, S, SW, ...) to the linear point relations (<, =, >) for both the horizontal and vertical axes. This is done in the function, <i>relative_position</i>, defined below.

Each Four2DPointNet represents a 4x4 matrix, where the 2x2 partition in the upper-right represents the relationship between two rectangles, A & B. The four elements in that partition can be mapped to an existing key (e.g., '>,<,>,<') used in the dictionary <i>qualreas.signature_name_mapping</i>, which then maps to an interval name.

In [ ]:
def relative_position(pt_rel):
    vert_pos = None
    horiz_pos = None

    if pt_rel in ["NW",  "N", "NE"]:
        vert_pos = ">"
    elif pt_rel in ["SW",  "S", "SE"]:
        vert_pos = "<"
    else:
        vert_pos = "="
            
    if pt_rel in ["E", "NE", "SE"]:
        horiz_pos = ">"
    elif pt_rel in ["W", "NW", "SW"]:
        horiz_pos = "<"
    else:
        horiz_pos = "="
    
    return horiz_pos, vert_pos

# This function is not used
def get_signature (net):
    arr = net.to_list()
    return arr[0][2], arr[0][3], arr[1][2], arr[1][3]

def get_keys(net_dict, ptalg2d_key):
    """Given a dictionary of consistent Four2DPointNet's, net_dict, like that output
    by 'generate_consistent_2D_networks', return the horizontal and vertical keys
    corresponding to interval relations.
    
    EXAMPLE: 
    result_proper = generate_consistent_2D_networks(ptalg2d, lessthan=["SW"], verbose=False)
    get_keys(result_proper, 'SE,SW,NE,SW') ==> ('>,<,>,<', '<,<,>,<')
    """
    ptnet = net_dict[ptalg2d_key]
    arr = ptnet.to_list()
    dirs2d = [arr[0][2], arr[0][3], arr[1][2], arr[1][3]]
    pos = list(map(relative_position, dirs2d))
    hkey = ','.join(map(lambda x: x[0], pos))
    vkey = ','.join(map(lambda x: x[1], pos))
    return hkey, vkey

def signature_name_2d(net_dict, ptalg2d_key, delimiter=":"):
    xy_keys = get_keys(net_dict, ptalg2d_key)
    x = qr.signature_name_mapping[xy_keys[0]]
    y = qr.signature_name_mapping[xy_keys[1]]
    return delimiter.join([x, y])

In [ ]:
print("2D (horiz_rel, vert_rel):")
print("-------------------------")
for rel in ptalg2d.elements:
    print(rel, relative_position(rel))

In [ ]:
ptalg2d_key = 'SE,SW,NE,SW'

In [ ]:
ptnet = result_proper[ptalg2d_key]
ptnet.to_list()

In [ ]:
signature_name_2d(result_proper, ptalg2d_key)

The result 'D:0', above, is illustrated in the figure below.

Wrt the figure, the meaning is how rectangle A relates to rectangle B. That is, A D:0 B.

In [ ]:
from IPython.display import Image
display(Image(filename='IMG_2285.jpeg', width=600))

Now, make sure that every Four2DPointNet can be assigned a name:

In [ ]:
all_proper_2d_names = [signature_name_2d(result_proper, key) for key in result_proper.keys()]
print(all_proper_2d_names)

In [ ]:
len(all_proper_2d_names)

How about doing the same for the extended rectangle definitions (includes horizontal segments, vertical segments, and points)?

In [ ]:
all_2d_names = [signature_name_2d(result_all, key) for key in result_all.keys()]
print(all_2d_names)

In [ ]:
len(all_2d_names)

In [ ]:
18**2